# 4元数クラス

In [1]:
# from pyquotenion import Quotanion

## クラス定義

In [2]:
import re

class quaternion:
    '''Quaternion class'''
    
    parts = ['r', 'i', 'j', 'k']
    
    def __init__(self, vec = (0,0,0,0)):
        self.coeffs = {'r':0, 'i':0, 'j':0, 'k':0}
        for i in range(len(self.parts)):
            self.coeffs[self.parts[i]] = vec[i]
    
    def __add__(self, q):
        if type(q) != type(self):
            raise ValueError("Can't add {}".format(type(q)))
        vec = tuple([self.get_coeff(p) + q.get_coeff(p) for p in self.parts])
        return type(self)(vec)
    
    def get_coeff(self, p):
        return self.coeffs[p]
    
    def show(self):
        mstr = '+'.join([str(self.get_coeff(p)) + p for p in self.parts])
        mstr = mstr.replace('r','')                      # 2r+3i --> 2+3i
        mstr = re.sub(r'1([ijk])', r'\1', mstr)          # 1+1i --> 1+i
        mstr = re.sub(r'(\D|^)0([ijk]*)', r'\1', mstr)   # 0+2i --> +2i, 2+0i+3j --> 2++3j
        mstr = re.sub(r'([+])[-+]+',r'\1', mstr)         # 1+-3i --> 1-3i, 1++3i --> 1+3i
        mstr = re.sub(r'^[+]|[+]$','', mstr)             # +2i --> 2i
        return mstr if mstr != '' else '0'

## テスト

In [3]:
import unittest
from IPython.display import Markdown, display

def run_check(test_class):
    t = test_class
    try:
        t.check()
    except t.failureException as e:
        display(Markdown('**<span style="color: red;">FAILED</span>**'))
        print('Hint:',  e)
    else: display(Markdown('**<span style="color: green;">PASSED</span>**'))

### メソッド ```show``` のテスト

In [4]:
class show_test(unittest.TestCase):
    pairs = ( 
        ((0,0,0,0),'0'),
        ((1,2,3,4),'1+2i+3j+4k'),
        ((1,1,1,1),'1+i+j+k'),
        ((-1,0,3,1),'-1+3j+k'),
#         ((0,-2,1,-3),'-2i+j-3k')
        )
    
    def check(self):
        for v, s in self.pairs:
            q = quaternion(v)
            self.assertEqual(q.show(), s)
            
run_check(show_test())

**<span style="color: green;">PASSED</span>**

In [5]:
import pytest

vec = (1,2,-3,1)
q1 = quaternion(vec)
# print(type(q1))
# q1.show()
assert type(q1) == quaternion
assert q1.show() == '1+2i+3j+k'

In [6]:
qi, qj, qk = quaternion((0,1,0,0)), quaternion((0,0,1,0)), quaternion((0,0,0,1))
(qi.show(), qj.show(), qk.show())

('i', 'j', 'k')

### メソッド ```__add__``` のテスト

In [7]:
class add_test(unittest.TestCase):
    vecs = [(0,0,0,0), (1,0,0,0), (0,1,0,0), (0,0,1,0), (0,0,0,1), (1,-2,-1,0)]
    pairs = ( ((0,0), (0,0,0,0)),
            ((0,1), vecs[0]),
            ((0,5), vecs[5]),
            ((3,5), (1,-2,0,0))
             )
       
    def check(self):
        for p, v in self.pairs:
            s = quaternion(self.vecs[p[0]]) + quaternion(self.vecs[p[1]])
            self.assertEqual(s.coeffs, v)
            
run_check(show_test())

**<span style="color: green;">PASSED</span>**